In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score, f1_score

In [2]:
df = pd.read_csv("/content/train.csv")
df_dev = pd.read_csv("/content/dev.csv")
df_test = pd.read_csv("/content/test.csv")

In [3]:
x = df.text
y = df.is_humor

dev_X = df_dev.text
dev_Y = df_dev.is_humor

gold_test_X = df_test.text
gold_test_Y = df_test.is_humor


In [4]:
Encoder = LabelEncoder()
y = Encoder.fit_transform(df.is_humor)
dev_Y = Encoder.fit_transform(df_dev.is_humor)
gold_test_Y = Encoder.fit_transform(df_test.is_humor)

Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(df.text)

x = Tfidf_vect.transform(df.text)
dev_X = Tfidf_vect.transform(df_dev.text)
gold_test_X = Tfidf_vect.transform(df_test.text)

In [5]:
def train(model):
  
  skf = model_selection.StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
  kFoldValFS = []
  devDataFS = []
  goldTestFS = []
  
  for train_index, test_index in skf.split(x, y):
    train_X, test_X = x[train_index], x[test_index]
    train_Y, test_Y = y[train_index], y[test_index]
    model.fit(train_X, train_Y)
    kFoldValFS.append(f1_score(test_Y, model.predict(test_X)))
    devDataFS.append(f1_score(dev_Y, model.predict(dev_X)))
    goldTestFS.append(f1_score(gold_test_Y, model.predict(gold_test_X)))

  print("kFoldValFS")
  print(kFoldValFS)
  print(np.mean(kFoldValFS))

  print("\ndevDataFS")
  print(devDataFS)
  print(np.mean(devDataFS))
  
  print("\ngoldTestFS")
  print(goldTestFS)
  print(np.mean(goldTestFS))

In [6]:
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
print("SVM")
train(SVM)

print("\nNaive Bayes")
Naive = naive_bayes.MultinomialNB()
train(Naive)

SVM
kFoldValFS
[0.8717434869739479, 0.881118881118881, 0.8864432216108054, 0.8834903321764999, 0.8690890990542558]
0.8783770041868781

devDataFS
[0.8456883509833585, 0.8536770280515542, 0.8582375478927203, 0.8470948012232415, 0.8490421455938697]
0.8507479747489487

goldTestFS
[0.8508914100486223, 0.852512155591572, 0.8520032706459526, 0.8487804878048779, 0.8488278092158448]
0.850603026661374

Naive Bayes
kFoldValFS
[0.8695652173913043, 0.8734995383194828, 0.8824895494658616, 0.875979714153988, 0.8688449148642429]
0.8740757868389759

devDataFS
[0.8402203856749312, 0.8488210818307905, 0.8500000000000001, 0.8463128876636803, 0.841522491349481]
0.8453753693037764

goldTestFS
[0.8649851632047478, 0.8715664439495173, 0.8707280832095097, 0.8673091178650851, 0.8636698599852617]
0.8676517336428242
